# Hill & Streaming Plots

## Read data

In [ ]:
import sys
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import dustpy
import dustpy.constants as c

if '..' not in sys.path:
    sys.path.append('..')

import helper
import plot_helper

In [ ]:
data_dir = Path('~').expanduser() / 'DATA/araa_data/lowerres_dipsy6/data_bump'

reader = dustpy.hdf5writer()
reader.datadir = str(data_dir)

# find time of strongest concentration
times = reader.read.sequence('t').squeeze() / c.year
rho = reader.read.sequence('dust.rho')
r = reader.read.sequence('grid.r')[0]
ir0 = np.abs(r - 20 * c.au).argmin()
#it = rho.sum(-1)[:, ir0:-10].max(-1).argmax()
it = np.abs(times - 3e6).argmin()

data = reader.read.output(it)

In [ ]:
f, ax = plt.subplots()

#ax.semilogx(times, r[rho.sum(-1)[:, ir0:-10].argmax(-1)+ir0]/ c.au)
ax.loglog(times, rho.sum(-1)[:, ir0:-10].max(-1), 'x')
tmax = times[rho.sum(-1)[:, ir0:-10].max(-1).argmax()]
ax.axvline(tmax)
ax.set_title(f'{tmax / 1e6:.2f} Myr')

## Hill Plot

In [ ]:
def hill_plot(axs=None):
    rho_d_nosedi = (data.dust.Sigma / np.sqrt(2*np.pi * data.gas.Hp[:, None]**2)).sum(-1)
    #rho_roche = 16 * 3 / (4 * np.pi) * data.star.M / data.grid.r**3
    rho_hill = 9 / (4 * np.pi) * data.star.M / data.grid.r**3


    if axs is None:
        f, axs = plt.subplots(2, 1, figsize=(6, 6), gridspec_kw={'height_ratios':[2,1], 'hspace':0}, sharex=True)
    else:
        f = axs[0].figure
    
    ax = axs[0]
    
    ax.loglog(data.grid.r / c.au, rho_d_nosedi, '-', label='mid-plane dust density\n(without sedimentation)')
    ax.loglog(data.grid.r / c.au, data.dust.rho.sum(-1), label='mid-plane dust density\n(with sedimentation)')
    ax.loglog(data.grid.r / c.au, data.dust.rho.sum(-1), '.', mfc='k', mec='k', ms=2)
    
    ax.loglog(data.grid.r / c.au, data.gas.rho, '--', label='mid-plane gas density')
    ax.loglog(data.grid.r / c.au, rho_hill, 'k', label=r'Hill density $\rho_\mathrm{H}$')

    ax.fill_between(data.grid.r / c.au, rho_hill, 1e-7*np.ones_like(data.grid.r), fc='k', alpha=0.25)
    
    ax.set_xlim(3, 100)
    ax.set_ylim(1e-18, 1e-8)
    ax.text(0.5, 0.95, f't = {times[it] / 1e6:.2f} Myr', transform=ax.transAxes, ha='center')
    
    # compute angle of the r^-3 lines
    pos = ax.get_position()
    angle = np.rad2deg(np.arctan(-3  / np.diff(np.log(ax.get_ylim())) * np.diff(np.log(ax.get_xlim()))) * \
                       f.get_size_inches()[1]/f.get_size_inches()[0] * \
                       pos.height / pos.width
                      )[0]

    ax.text(10, 7e-12, r'$\uparrow$ gravitational collapse $\uparrow$', rotation=angle, c='k')
    #t = ax.text(28, 5e-12, 'text',
    #        ha="center", va="center", rotation=90, size=10,
    #        bbox=dict(boxstyle="rarrow,pad=0.1", fc="gray", ec="none", lw=2))
    
    # add the Hill limits
    for i in range(1, 5):
        ax.text(5, 7e-10 * 10**-i, fr'$10^{{{-i}}} \cdot \rho_\mathrm{{H}}$', rotation=angle, c='0.7')
        ax.loglog(data.grid.r / c.au, rho_hill * 10**-i, c='0.7', ls=':')
    
    ax.legend(fontsize='small', loc='lower left')
    ax.set_ylabel('density $\\rho$ [g cm$^{-3}$]')
    ax.set_facecolor('none')
    
    ax = axs[1]
    
    a = data.dust.a[0, :]
    csum = np.cumsum(data.dust.Sigma, 1) / data.dust.Sigma.sum(-1)[:, None]
    amax = [np.interp(0.9, csum[ir,:], a) for ir in range(len(r))]
    Stmax = [np.interp(amax[ir], a, data.dust.St[ir, :]) for ir in range(len(r))]
    
    
    #ax.contour(data.grid.r / c.au, data.dust.a[0, :], data.dust.St.T, [1], linestyles='--', colors='k')
    #ax.contour(data.grid.r / c.au, data.dust.a[0, :], , [0.95], linestyles='-', colors='k')
    ax.plot(data.grid.r / c.au, Stmax, 'k', label='$\mathrm{St}_\mathrm{max}$')
    ax.plot(data.grid.r / c.au, (data.dust.v.frag/data.gas.cs)**2 / (3 * data.gas.alpha), 'k--', label='$\mathrm{St}_\mathrm{frag}$')
    ax.legend(fontsize='small')
    ax.set_yscale('log')
    ax.set_ylim(3e-3, 3)
    ax.set_xlabel('distance to star $r$ [au]')
    ax.grid(axis='y', ls='--')
    ax.set_ylabel('Stokes number')

hill_plot()

## Streaming criterion plot

In [ ]:
def eps_crit(St):
    lg_St = np.log10(St)
    mask = St < 0.015
    A = np.where(mask, 0.0, 0.48)
    B = np.where(mask, 0.0, 0.87)
    C = np.where(mask, np.log10(2.5), -0.11)
    return 10.**(A * lg_St**2 + B * lg_St + C)

def Z_crit(St, alpha, PI=0.05):
    return eps_crit(St) * np.sqrt((PI / 5)**2 + alpha / (alpha + St))

In [ ]:
import matplotlib.transforms as mtransforms

def plot_streaming_criterion(ax=None):
    if ax is None:
        f, ax = plt.subplots(figsize=(6, 4), dpi=150)
    else:
        f = ax.figure

    St = np.geomspace(3e-4, 1e1, 100)
    ax.set_xlim(St[[0, -1]])
    ax.set_ylim(1e-3, 1)
    pos = ax.get_position()
    
    # use data coordinates for the x-axis and the axes coordinates for the y-axis
    trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
    
    cs = np.sqrt(c.k_B * 30 /(2.3 * c.m_p))
    
    for i, alpha in enumerate([0, 1e-4, 1e-3, 1e-2]):
        Zc = Z_crit(St, alpha)
        ZD = np.sqrt(alpha / (alpha + St))
    
        if alpha > 0:
            St_f0 = (100/cs)**2 / (3 * alpha)
            St_f1 = (1000/cs)**2 / (3 * alpha)
    
            label = rf'$\alpha = 10^{{{np.log10(alpha)}}}$'
    
            # draw the lines
            mask = (St >= St_f0) & (St<=St_f1)
            line1, = ax.loglog(St, Zc, alpha=0.5)
            ax.loglog(St[mask], Zc[mask], c=line1.get_color(), label=label)
            
            line2, = ax.loglog(St, ZD, '--', c=line1.get_color(), alpha=line1.get_alpha())
            ax.loglog(St[mask], ZD[mask], '--', c=line1.get_color())
    
            # add the fill between & text
            opts=dict(transform=trans, va='center', c=line1.get_color())
            ax.fill_between([St_f0, St_f1], 0.1 * (i-1), 0.1 * i, fc=line1.get_color(), alpha=0.5, transform=trans)
            ax.text(St_f0, 0.1 * (i-1) +  0.05, '1 m/s', rotation=90, ha='left', fontsize='xx-small', **opts)
            ax.text(St_f1, 0.1 * (i-1) +  0.05, '10 m/s', rotation=90, ha='right', fontsize='xx-small', **opts)
            ax.text(np.sqrt(St_f0*St_f1), 0.1 * (i-1) +  0.05, '$\mathrm{St}_\mathrm{frag}$', ha='center', fontsize='small', **opts)
        else:
            label = fr'$\alpha = {alpha}$'
    
            # draw the lines
            line1, = ax.loglog(St, Zc, label=label)
            ax.loglog(St, ZD, '--', c=line1.get_color())
                   
    
    ax.plot([], 'k-', label='$Z_{LY21}$')
    ax.plot([], 'k--', label='$\epsilon_\mathrm{mid} = 1$')
    
    ax.legend(fontsize='small', loc='upper right')
    ax.set_xlabel('St')
    ax.set_ylabel('$Z_\mathrm{crit}$')

plot_streaming_criterion()

## Both together

In [ ]:
f = plt.figure(figsize=(12, 5), constrained_layout=True)
gs = f.add_gridspec(2, 2, height_ratios=[2,1])

ax1 = f.add_subplot(gs[0, 0])
ax2 = f.add_subplot(gs[1, 0])
ax3 = f.add_subplot(gs[:, 1])

ax1.sharex(ax2)

hill_plot(axs=[ax1, ax2])
plot_streaming_criterion(ax=ax3)

for ax in [ax1, ax2, ax3]:
    helper.apply_araa_style(ax)

f.savefig(helper.output_dir / 'BirnstielFig11.pdf', transparent=True, bbox_inches='tight')